# метод Шерлока Холмса

Условие задания ([ссылка](https://docs.google.com/document/d/1pA1lK1pxWAniSFCp6870gwfjhtoLKZEg_DOXwW0truA/edit))

In [1]:
from nltk import ngrams
import nltk

import re
import typing as tp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
RU_ALPHABET = "абвгдежзийклмнопрстуфхцчшщъыьэюя "
ENG_ALPHABET = "abcdefghijklmnopqrstuvwxyz "
SPEC = " "

In [3]:
corpus = []
with open("AnnaKarenina.txt", "r", encoding='utf-8') as fin:
    corpus += fin.readlines()
with open("WarAndPeace.txt", "r", encoding='utf-8') as fin:
    corpus += fin.readlines()

clear_corpus = []
for line in corpus:
    new_line = []
    for symbol in line:
        symbol = symbol.lower()
        if symbol in RU_ALPHABET or symbol in SPEC:
            new_line += [symbol]
    new_line = "".join(new_line)
    new_line = re.sub(' +', ' ', new_line)
    if len(new_line) > 5:
        clear_corpus += [new_line]

In [4]:
eng_corpus = []
with open("WarAndPeaceEng.txt", "r", encoding='utf-8') as fin:
    eng_corpus += fin.readlines()

clear_eng_corpus = []
for line in eng_corpus:
    new_line = []
    for symbol in line:
        symbol = symbol.lower()
        if symbol in ENG_ALPHABET or symbol in SPEC:
            new_line += [symbol]
    new_line = "".join(new_line)
    new_line = re.sub(' +', ' ', new_line)
    if len(new_line) > 5:
        clear_eng_corpus += [new_line]

In [5]:
def encrypt_corpus(corpus: tp.List, ru2rnd: dict) -> tp.List:
    encrypted = []   
    
    n_gramm = len(list(ru2rnd.items())[0][0])
    
    for line in corpus:
        new_line = None
        for i in range(len(line) - n_gramm + 1):
            gram = tuple(letter for letter in line[i: i + n_gramm])
            
            if new_line is None:
                new_line = "".join(ru2rnd[gram])
            elif n_gramm == 1:
                new_line += "".join(ru2rnd[gram])
            else:
                new_line = new_line[:-n_gramm + 1] + "".join(ru2rnd[gram])

        encrypted += [new_line]
    
    
    return encrypted

In [6]:
def corpuses_accuracy(corpus1: tp.List, corpus2: tp.List) -> float:
    len1 = sum(
        len(line) for line in corpus1
    )
    
    len2 = sum(
        len(line) for line in corpus2
    )
    
    if len1 != len2:
        raise ValueError("Texts must be the same size")
    
    true_count = sum([
        sum([sym1 == sym2 for sym1, sym2 in zip(line1, line2)])
        for line1, line2 in zip(corpus1, corpus2)
    ])

    
    return true_count / len1

In [7]:
def ngramm_freqs(corpus: tp.List, n=1, return_freqs = True) -> dict:
    tokens = "".join(corpus)

    #Create your bigrams
    ngs = ngrams(tokens, n)

    #compute frequency distribution for all the bigrams in the text
    fdist = nltk.FreqDist(ngs)

    if not return_freqs:
        return {
        letter: count for letter, count in sorted(fdist.items(), key=lambda item: -item[1])
    }
    
    total_count = 0
    for _, v in fdist.items():
        total_count += v
        
    freqs = {
        letter: (count / total_count) for letter, count in sorted(fdist.items(), key=lambda item: -item[1])
    }
    
    return freqs

In [8]:
rnd_idx = np.random.choice(len(RU_ALPHABET), len(RU_ALPHABET), replace=False)

ru2rnd = {
    (RU_ALPHABET[i], ): (RU_ALPHABET[rnd_idx[i]], ) for i in range(len(RU_ALPHABET))
}

In [9]:
test_corpus = np.random.choice(clear_corpus, 2).tolist()
encrypted_test_corpus = encrypt_corpus(test_corpus, ru2rnd)

## Часть 1

In [10]:
n_gramm = 1

total_freqs = ngramm_freqs(clear_corpus, n_gramm)
encrypted_test_freqs = ngramm_freqs(encrypted_test_corpus, n_gramm)

enc2ru = {
    enc: ru for (enc, _), (ru, _) in zip(encrypted_test_freqs.items(), total_freqs.items())
}

decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, enc2ru)


print("Текст для проверки:   ", test_corpus)
print("Зашифрованный текст:  ", encrypted_test_corpus)
print("Расшифрованный текст: ", decoded_test_corpus)
print("Доля правильных букв: ", corpuses_accuracy(test_corpus, decoded_test_corpus))

Текст для проверки:    [' где вы поместите княгиню сказал вронский пофранцузски обращаясь к анне и не дождавшись ответа еще раз поздоровался с дарьей александровной и теперь поцеловал ее руку я думаю в большой балконной', ' пари проиграно мы прежде приехали расплачивайтесь говорил он улыбаясь']
Зашифрованный текст:   ['дслрднйдвьщряиаирдмзксазгдямъ ъхдныьзямаудвьюыъзфб ямадьчыътъкяэдмдъззрдадзрдльшлъноаяэдьинриъдртрдыъ двь льыьнъхякдядлъыэрудъхрмяъзлыьнзьудадирврыэдвьфрхьнъхдррдыбмбдкдлбщъгдндчьхэоьудчъхмьззьу', 'двъыадвыьасыъзьдщйдвыршлрдвыаржъхадыъявхъпанъуиряэдсьньыахдьздбхйчъкяэ']
Расшифрованный текст:  [' гка вз речасуиуа дтягитж сдоыол внетсдип реэнотйьысди ебношоясм д отта и та кехковюисм еувауо аша ноы реыкеневолся с конмап оладсоткневтеп и уаранм рейалевол аа ньдь я кьчож в белмюеп болдеттеп', ' рони рнеигноте чз рнахка рниацоли носрлощивопуасм гевенил ет ьлзбоясм']
Доля правильных букв:  0.3712121212121212


## Часть 2

In [11]:
n_gramm = 2

total_freqs = ngramm_freqs(clear_corpus, n_gramm)
encrypted_test_freqs = ngramm_freqs(encrypted_test_corpus, n_gramm)

enc2ru = {
    enc: ru for (enc, _), (ru, _) in zip(encrypted_test_freqs.items(), total_freqs.items())
}

decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, enc2ru)


print("Текст для проверки:   ", test_corpus)
print("Зашифрованный текст:  ", encrypted_test_corpus)
print("Расшифрованный текст: ", decoded_test_corpus)
print("Доля правильных букв: ", corpuses_accuracy(test_corpus, decoded_test_corpus))

Текст для проверки:    [' где вы поместите княгиню сказал вронский пофранцузски обращаясь к анне и не дождавшись ответа еще раз поздоровался с дарьей александровной и теперь поцеловал ее руку я думаю в большой балконной', ' пари проиграно мы прежде приехали расплачивайтесь говорил он улыбаясь']
Зашифрованный текст:   ['дслрднйдвьщряиаирдмзксазгдямъ ъхдныьзямаудвьюыъзфб ямадьчыътъкяэдмдъззрдадзрдльшлъноаяэдьинриъдртрдыъ двь льыьнъхякдядлъыэрудъхрмяъзлыьнзьудадирврыэдвьфрхьнъхдррдыбмбдкдлбщъгдндчьхэоьудчъхмьззьу', 'двъыадвыьасыъзьдщйдвыршлрдвыаржъхадыъявхъпанъуиряэдсьньыахдьздбхйчъкяэ']
Расшифрованный текст:  [' лнинвло рзлнтссирто лакйеьо ранни адьвтео ае товаыьв  вд ачмр яр  тоуип  уиноб л жес я  иетидоаеиг  ло овот  паю еемнлеочое аас стнс  нкоеп мсмхиояо  киэ паномигмттааеннрсвйнываеесиоев аоиаокон', 'о ек оо дшх ткечплооан ниоокб бае г зосидлиписсл я с уткдн ашиоиу мр о']
Доля правильных букв:  0.09090909090909091


Получилось, что униграммы решают задачу много лучше, чем биграммы. Я склонен к выводу, что это связанно исключительно из-за наличия пробелом в символах для подсчета частот.

## Часть 3

In [12]:
def loglikelihood(reference_bigramm_count, decoded_bigramm_count):
    lll = 0
    for decoded_bigramm, decoded_count in decoded_bigramm_count.items():
        # Ко всем биграммам, которые не встретились в тексте,
        # добавим Байесовское сглаживание, чтобы не обнулять lll
        if decoded_bigramm in reference_bigramm_count.keys():
            lll += decoded_count * np.log(reference_bigramm_count[decoded_bigramm])
    return lll

In [13]:
test_corpus = np.random.choice(clear_corpus, 10).tolist()
encrypted_test_corpus = encrypt_corpus(test_corpus, ru2rnd)
reference_bigramm_count = ngramm_freqs(clear_corpus, 2, return_freqs=False)

In [14]:
chiper_idx = np.random.choice(len(RU_ALPHABET), len(RU_ALPHABET), replace=False)

chiper = {
    (RU_ALPHABET[i], ): (RU_ALPHABET[chiper_idx[i]], ) for i in range(len(RU_ALPHABET))
}


iter_num = 30000

decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, chiper)
decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, 2, return_freqs=False)
curr_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)

for iteration in range(iter_num):
    
    keys_to_swap = np.random.choice(list(RU_ALPHABET), 2, replace=False)
    new_chiper = chiper.copy()
    new_chiper[(keys_to_swap[0], )] = chiper[(keys_to_swap[1], )]
    new_chiper[(keys_to_swap[1], )] = chiper[(keys_to_swap[0], )]
    
    decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, new_chiper)
    decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, 2, return_freqs=False)
    new_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)
    
    if new_lll > curr_lll:
        chiper = new_chiper
        curr_lll = new_lll
        
    elif np.random.uniform(0, 1) < np.exp(new_lll - curr_lll):
        chiper = new_chiper
        curr_lll = new_lll

In [15]:
decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, chiper)
print("Текст для проверки:   ", test_corpus)
print("Зашифрованный текст:  ", encrypted_test_corpus)
print("Расшифрованный текст: ", decoded_test_corpus)
print("Доля правильных букв: ", corpuses_accuracy(test_corpus, decoded_test_corpus))

Текст для проверки:    ['наташа оторвав на минуту лицо от кружевной косынки матери взглянула на нее снизу сквозь слезы смеха и опять спрятала лицо', 'косые лучи солнца были еще жарки платье насквозь промокшее от пота липло к телу левый сапог полный воды был тяжел и чмокал по испачканному пороховым осадком лицу каплями скатывался пот во рту была горечь в носу запах пороха и ржавчины в ушах неперестающее чмоканье бекасов до стволов нельзя было дотронуться так они разгорелись сердце стучало быстро и коротко руки тряслись от волнения и усталые ноги спотыкались и переплетались по кочкам и трясине но он все ходил и стрелял наконец сделав постыдный промах он бросил наземь ружье и шляпу', ' ну полно вам проговорил аудитор с сияющею наивною и вместе хитрою улыбкой как будто ему лестно было что он составлял предмет шуток жеркова и как будто он нарочно старался казаться глупее чем он был в самом деле', ' я знал я знал повторял он свою любимую фразу и схватив ее руку которая ласкала его волосы ста

In [16]:
mistakes = {
    key: (value, ru2rnd[value]) for key, value in chiper.items() if key != ru2rnd[value] 
}
len(mistakes), mistakes

(0, {})

## Часть 4

In [17]:
text_to_decode = ["←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"]
len(set(text_to_decode[0])) # Скорее всего это английский алфовит, а не русский

28

In [18]:
chiper_idx = np.random.choice(len(ENG_ALPHABET), len(ENG_ALPHABET), replace=False)

chiper = {
    (sym, ): 
        (ENG_ALPHABET[chiper_idx[i]], ) if i < len(ENG_ALPHABET) 
        else (".", )
        for i, sym in enumerate(set(text_to_decode[0]))
}

iter_num = 100000

decoded_test_corpus = encrypt_corpus(text_to_decode, chiper)
decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, 2, return_freqs=False)
reference_bigramm_count = ngramm_freqs(clear_eng_corpus, 2, return_freqs=False)
curr_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)

for iteration in range(iter_num):
    
    keys_to_swap = np.random.choice(list(set(text_to_decode[0])), 2, replace=False)
    new_chiper = chiper.copy()
    new_chiper[(keys_to_swap[0], )] = chiper[(keys_to_swap[1], )]
    new_chiper[(keys_to_swap[1], )] = chiper[(keys_to_swap[0], )]
    
    decoded_test_corpus = encrypt_corpus(text_to_decode, new_chiper)
    decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, 2, return_freqs=False)
    new_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)
    
    if new_lll > curr_lll:
        chiper = new_chiper
        curr_lll = new_lll
        
    elif np.random.uniform(0, 1) < np.exp(new_lll - curr_lll):
        chiper = new_chiper
        curr_lll = new_lll

In [ ]:
decoded_test_corpus = encrypt_corpus(text_to_decode, chiper)
print("Зашифрованный текст:  ", text_to_decode)
print("Расшифрованный текст: ", decoded_test_corpus)

Зашифрованный текст:   ['←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏']
Расшифрованный текст:  ['eisa cl cagane horbusthly asa fowna horbusthly nemin p .novo iookxehad monorly sevmo frowanunt imoree cievo cl cie igesusa frucastho a fospwane bumiabusthly kuss qu foiseghee wencernoe quguhae mpriu jond mohewho d hawevo he okexuz']


Что-то тут совсем плохо, может это русский язык, просто не все символы в тексте присутствуют

In [20]:
most_freq_symbols = list(ngramm_freqs(clear_corpus, 1).keys())

epoch_num = 20

best_chiper = None
best_lll = 0

for epoch in range(epoch_num):

    chiper = {
        (sym, ): most_freq_symbols[i] for i, sym in enumerate(set(text_to_decode[0]))
    }

    iter_num = 5000

    decoded_test_corpus = encrypt_corpus(text_to_decode, chiper)
    decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, 2, return_freqs=False)
    reference_bigramm_count = ngramm_freqs(clear_corpus, 2, return_freqs=False)
    curr_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)

    for iteration in range(iter_num):
        
        keys_to_swap = np.random.choice(list(set(text_to_decode[0])), 2, replace=False)
        
        new_chiper = chiper.copy()
        new_chiper[(keys_to_swap[0], )] = chiper[(keys_to_swap[1], )]
        new_chiper[(keys_to_swap[1], )] = chiper[(keys_to_swap[0], )]
        
        decoded_test_corpus = encrypt_corpus(text_to_decode, new_chiper)
        decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, 2, return_freqs=False)
        new_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)
        
        if new_lll > curr_lll:
            chiper = new_chiper
            curr_lll = new_lll
            
        elif np.random.uniform(0, 1) < np.exp(new_lll - curr_lll):
            chiper = new_chiper
            curr_lll = new_lll
    
    if curr_lll > best_lll:
        best_lll = curr_lll
        best_chiper = chiper

Результаты оказались слишком шумными, поэтому решил запустить алгоритм Метрополиса несколько раз и сохранить лучшую по loglikelihood перестановку

In [21]:
decoded_test_corpus = encrypt_corpus(text_to_decode, best_chiper)
print("Зашифрованный текст:  ", text_to_decode)
print("Расшифрованный текст: ", decoded_test_corpus)

Зашифрованный текст:   ['←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏']
Расшифрованный текст:  ['если вы вимите норзальный или подти норзальный текст у чтого сообшения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный балл жа послемнее детвертое жамание курса хотя конедно я нидего не обешаю']


Лучшая расшифровка, которая получилось:

* если вы вимите норжальный или подти норжальный текст у чтого сообшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный балл за послемнее детвертое замание курса хотя конедно я нидего не обешаю

## Часть 5

In [22]:
test_corpus = np.random.choice(clear_corpus, 10).tolist()
encrypted_test_corpus = encrypt_corpus(test_corpus, ru2rnd)

In [23]:
def decode_with_n_gramm(test_corpus, encrypted_test_corpus, n_gramm):
    chiper_idx = np.random.choice(len(RU_ALPHABET), len(RU_ALPHABET), replace=False)

    chiper = {
        (RU_ALPHABET[i], ): (RU_ALPHABET[chiper_idx[i]], ) for i in range(len(RU_ALPHABET))
    }

    iter_num = 3000

    decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, chiper)
    decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, n_gramm, return_freqs=False)
    reference_bigramm_count = ngramm_freqs(clear_corpus, n_gramm, return_freqs=False)
    curr_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)

    for iteration in range(iter_num):
        
        keys_to_swap = np.random.choice(list(RU_ALPHABET), 2, replace=False)
        new_chiper = chiper.copy()
        new_chiper[(keys_to_swap[0], )] = chiper[(keys_to_swap[1], )]
        new_chiper[(keys_to_swap[1], )] = chiper[(keys_to_swap[0], )]
        
        decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, new_chiper)
        decoded_bigramm_count = ngramm_freqs(decoded_test_corpus, n_gramm, return_freqs=False)
        new_lll = loglikelihood(reference_bigramm_count, decoded_bigramm_count)
        
        if new_lll > curr_lll:
            chiper = new_chiper
            curr_lll = new_lll
            
        elif np.random.uniform(0, 1) < np.exp(new_lll - curr_lll):
            chiper = new_chiper
            curr_lll = new_lll
    
    decoded_test_corpus = encrypt_corpus(encrypted_test_corpus, chiper)
    print("Текст для проверки:   ", test_corpus)
    print("Зашифрованный текст:  ", encrypted_test_corpus)
    print("Расшифрованный текст: ", decoded_test_corpus)
    print("Доля правильных букв: ", corpuses_accuracy(test_corpus, decoded_test_corpus))

In [30]:
decode_with_n_gramm(test_corpus, encrypted_test_corpus, n_gramm=3)

Текст для проверки:    ['он прикинул воображением места куда он мог бы ехать клуб партия безика шампанское с игнатовым нет не поеду там найду облонского куплеты нет надоело вот именно за то я люблю щербацких что сам лучше делаюсь поеду домой он прошел прямо в свой номер у дюссо велел подать себе ужинать и потом раздевшись только успел положить голову на подушку заснул крепким и спокойным как всегда сном', 'одна выгода этой городской жизни была та что ссор здесь в городе между ними никогда не было оттого ли что условия городские другие или оттого что они оба стали осторожнее и благоразумнее в этом отношении в москве у них не было ссор изза ревности которых они так боялись переезжая в город', ' да что скоро ли там кавалерия говорят дорогу загородила говорил офицер', ' пойдем ужинать сказал он со вздохом вставая и направляясь к двери', ' вы поедете на этот бал спросила кити', ' что делать наша должность такая у господ покойнее зато расчетов здесь больше', 'это обычно франц', 'сергей ивано

In [31]:
decode_with_n_gramm(test_corpus, encrypted_test_corpus, n_gramm=4)

Текст для проверки:    ['он прикинул воображением места куда он мог бы ехать клуб партия безика шампанское с игнатовым нет не поеду там найду облонского куплеты нет надоело вот именно за то я люблю щербацких что сам лучше делаюсь поеду домой он прошел прямо в свой номер у дюссо велел подать себе ужинать и потом раздевшись только успел положить голову на подушку заснул крепким и спокойным как всегда сном', 'одна выгода этой городской жизни была та что ссор здесь в городе между ними никогда не было оттого ли что условия городские другие или оттого что они оба стали осторожнее и благоразумнее в этом отношении в москве у них не было ссор изза ревности которых они так боялись переезжая в город', ' да что скоро ли там кавалерия говорят дорогу загородила говорил офицер', ' пойдем ужинать сказал он со вздохом вставая и направляясь к двери', ' вы поедете на этот бал спросила кити', ' что делать наша должность такая у господ покойнее зато расчетов здесь больше', 'это обычно франц', 'сергей ивано

In [26]:
decode_with_n_gramm(test_corpus, encrypted_test_corpus, n_gramm=5)

Текст для проверки:    ['он прикинул воображением места куда он мог бы ехать клуб партия безика шампанское с игнатовым нет не поеду там найду облонского куплеты нет надоело вот именно за то я люблю щербацких что сам лучше делаюсь поеду домой он прошел прямо в свой номер у дюссо велел подать себе ужинать и потом раздевшись только успел положить голову на подушку заснул крепким и спокойным как всегда сном', 'одна выгода этой городской жизни была та что ссор здесь в городе между ними никогда не было оттого ли что условия городские другие или оттого что они оба стали осторожнее и благоразумнее в этом отношении в москве у них не было ссор изза ревности которых они так боялись переезжая в город', ' да что скоро ли там кавалерия говорят дорогу загородила говорил офицер', ' пойдем ужинать сказал он со вздохом вставая и направляясь к двери', ' вы поедете на этот бал спросила кити', ' что делать наша должность такая у господ покойнее зато расчетов здесь больше', 'это обычно франц', 'сергей ивано

In [33]:
decode_with_n_gramm(test_corpus, encrypted_test_corpus, n_gramm=6)

Текст для проверки:    ['он прикинул воображением места куда он мог бы ехать клуб партия безика шампанское с игнатовым нет не поеду там найду облонского куплеты нет надоело вот именно за то я люблю щербацких что сам лучше делаюсь поеду домой он прошел прямо в свой номер у дюссо велел подать себе ужинать и потом раздевшись только успел положить голову на подушку заснул крепким и спокойным как всегда сном', 'одна выгода этой городской жизни была та что ссор здесь в городе между ними никогда не было оттого ли что условия городские другие или оттого что они оба стали осторожнее и благоразумнее в этом отношении в москве у них не было ссор изза ревности которых они так боялись переезжая в город', ' да что скоро ли там кавалерия говорят дорогу загородила говорил офицер', ' пойдем ужинать сказал он со вздохом вставая и направляясь к двери', ' вы поедете на этот бал спросила кити', ' что делать наша должность такая у господ покойнее зато расчетов здесь больше', 'это обычно франц', 'сергей ивано

In [34]:
decode_with_n_gramm(test_corpus, encrypted_test_corpus, n_gramm=10) # размер н-грамма ~ 1-2-ум словам

Текст для проверки:    ['он прикинул воображением места куда он мог бы ехать клуб партия безика шампанское с игнатовым нет не поеду там найду облонского куплеты нет надоело вот именно за то я люблю щербацких что сам лучше делаюсь поеду домой он прошел прямо в свой номер у дюссо велел подать себе ужинать и потом раздевшись только успел положить голову на подушку заснул крепким и спокойным как всегда сном', 'одна выгода этой городской жизни была та что ссор здесь в городе между ними никогда не было оттого ли что условия городские другие или оттого что они оба стали осторожнее и благоразумнее в этом отношении в москве у них не было ссор изза ревности которых они так боялись переезжая в город', ' да что скоро ли там кавалерия говорят дорогу загородила говорил офицер', ' пойдем ужинать сказал он со вздохом вставая и направляясь к двери', ' вы поедете на этот бал спросила кити', ' что делать наша должность такая у господ покойнее зато расчетов здесь больше', 'это обычно франц', 'сергей ивано

In [35]:
decode_with_n_gramm(test_corpus, encrypted_test_corpus, n_gramm=15) # размер н-грамма ~2-3-ём словам 

Текст для проверки:    ['он прикинул воображением места куда он мог бы ехать клуб партия безика шампанское с игнатовым нет не поеду там найду облонского куплеты нет надоело вот именно за то я люблю щербацких что сам лучше делаюсь поеду домой он прошел прямо в свой номер у дюссо велел подать себе ужинать и потом раздевшись только успел положить голову на подушку заснул крепким и спокойным как всегда сном', 'одна выгода этой городской жизни была та что ссор здесь в городе между ними никогда не было оттого ли что условия городские другие или оттого что они оба стали осторожнее и благоразумнее в этом отношении в москве у них не было ссор изза ревности которых они так боялись переезжая в город', ' да что скоро ли там кавалерия говорят дорогу загородила говорил офицер', ' пойдем ужинать сказал он со вздохом вставая и направляясь к двери', ' вы поедете на этот бал спросила кити', ' что делать наша должность такая у господ покойнее зато расчетов здесь больше', 'это обычно франц', 'сергей ивано

Для уменьшения времени работы, я уменьшил число итераций для данной части. Результаты достаточно шумные, но увеличенные н-граммы дали лучший результат за меньшее число итераций. 

Однако моё мнение, что нет смысла увеличивать н-граммы до размеров больших, чем размер в среднем слов. Если у меня будут н-граммы на пример +- размер двух слов, то такая статистика будет сильно обусловленна на тему документа. 

## Часть 6

Применения, которые приходят мне в голову:
* Расшифровка древних языков на языки, нам известные для того времени.
* Какое-то подобие перевода для наших дней 